In [ ]:
from datasets import load_dataset

In [ ]:
dirs = """
cmg_ga_mu_corpusbeag
pmg_ga_co/RCPiarsachALL
nnc_ga_mu/CDNamedEntities01
nnc_ga_mu/CDAn_tOileanach_part3_newLTS
nnc_ga_mu/CDAn_tOileanach_part4_newLTS
nnc_ga_mu/CDNamedEntities02
nnc_ga_mu/CDAlphabet
nnc_ga_mu/CDAn_tOileanach_part2_newLTS
nnc_ga_mu/corpas_beag_newLTS
nnc_ga_mu/CDSaytime
nnc_ga_mu/CDAn_tOileanach_part1_newLTS
cmg_ga_mu/CMGOther
cmg_ga_mu/CMGCorpusBeag
anb_ga_ul/new_text02
anb_ga_ul/text01-004
anb_ga_ul/text01
anb_ga_ul/text01-001
anb_ga_ul/text02
anb_ga_ul/named-entities
nnc_en_ie
"""

In [ ]:
dirl = dirs.split("\n")